In [1]:
#Importing libraries
import numpy as np
import math as m
import pandas as pd
import matplotlib.pyplot as plt

In [106]:
#Ask forfileinput (Fancy pendant)
#Import data from roads
InitialRoadData = pd.read_csv('..\RMMS\CSV\_allRoads.csv')
RoadData = InitialRoadData

In [107]:
#Deleting inconsistent coordinates 
Tolerance = 0.02
RoadData['UpperErrorLat']=abs(RoadData['Latitude'].shift(-1)-RoadData['Latitude'])>Tolerance
RoadData['BelowErrorLat']=abs(RoadData['Latitude'].shift()-RoadData['Latitude'])>Tolerance
RoadData['UpperErrorLon']=abs(RoadData['Longitude'].shift(-1)-RoadData['Longitude'])>Tolerance
RoadData['BelowErrorLon']=abs(RoadData['Longitude'].shift()-RoadData['Longitude'])>Tolerance
RoadData['RoadIDCheckAbove']=RoadData['Road Id']==RoadData['Road Id'].shift(-1)
RoadData['RoadIDCheckBelow']=RoadData['Road Id']==RoadData['Road Id'].shift()
RoadData['Error']=((RoadData['RoadIDCheckAbove'] & RoadData['RoadIDCheckBelow'] & ((RoadData['UpperErrorLat'] & RoadData['BelowErrorLat'])|(RoadData['UpperErrorLon'] & RoadData['BelowErrorLon'])))|
                   (RoadData['RoadIDCheckAbove'] & -RoadData['RoadIDCheckBelow'] & ((RoadData['UpperErrorLat'] &  -RoadData['UpperErrorLat'].shift(-1))|(RoadData['UpperErrorLon'] &  -RoadData['UpperErrorLon'].shift(-1))))|
                   (-RoadData['RoadIDCheckAbove'] & RoadData['RoadIDCheckBelow'] & ((RoadData['BelowErrorLat'] &  -RoadData['BelowErrorLat'].shift())|(RoadData['BelowErrorLon'] &  -RoadData['BelowErrorLon'].shift())))
                  )
ProcRoadData.loc[RoadData.Error == True, 'Latitude'] = np.NaN
ProcRoadData.loc[RoadData.Error == True, 'Longitude'] = np.NaN      
#Interpolating
SmoothRoadData = RoadData.interpolate()
del SmoothRoadData['UpperErrorLat']
del SmoothRoadData['UpperErrorLon']
del SmoothRoadData['BelowErrorLat']
del SmoothRoadData['BelowErrorLon']
del SmoothRoadData['RoadIDCheckAbove']
del SmoothRoadData['RoadIDCheckBelow']
del SmoothRoadData['Error']
SmoothRoadData

,Road Id,LRP Number,Road Chainage,Type,Desc,Latitude,Longitude
0,N1,LRPS,0.000,Others,Start of Road after Jatrabari Flyover infront...,23.706028,90.443333
1,N1,LRPSa,0.814,Culvert,Box Culvert,23.702917,90.450417
2,N1,LRPSb,0.822,Cross Road,Intersection with Z1101,23.702778,90.450472
3,N1,LRP001,1.000,Km Post,Km post missing,23.702139,90.451972
4,N1,LRP002,2.000,Km Post,Km post missing,23.697889,90.460583
5,N1,LRP002a,2.130,Culvert,Box culvert,23.697361,90.461667
6,N1,LRP003,3.000,Km Post,Km post missing,23.693833,90.469138
7,N1,LRP004,4.000,Km Post,Km post missing,23.693611,90.478777
8,N1,LRP004a,4.175,"Side Road, Right",Road to Narayanganj(R111),23.693805,90.480527
9,N1,LRP005,5.000,Km Post,Km post missing,23.694750,90.488500
